<a href="https://www.kaggle.com/code/golammostofas/function-calling-using-openai?scriptVersionId=219736204" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Overview

In this notebook, you will learn about fuction calling using Open AI.

I will excute a function(get_weather) for geting wheather. you can [get](https://api.open-meteo.com/v1/forecast?latitude=23.8103&longitude=90.4125&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m) weather information in Dhaka.

In [1]:
import requests


# this is the function which we will call
def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

# Get API Key

you have to need opean-ai api key

In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("OPENAI_API_KEY") # replace your openAI api key

# install requiremetns

In [3]:
!pip install openai

In [4]:
from openai import OpenAI

# Create a Client
client = OpenAI(api_key=secret_value_0)


In [5]:


tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for provided coordinates in celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }
}]

messages = [{"role": "user", "content": "What's the weather like in Dhaka today?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

completion

ChatCompletion(id='chatcmpl-AuwRs9yRa9XWvXVetfnNOyo62iJHG', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_cHPme3F2si8XlqzURMpW5FZA', function=Function(arguments='{"latitude":23.8103,"longitude":90.4125}', name='get_weather'), type='function')]))], created=1738134408, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_4691090a87', usage=CompletionUsage(completion_tokens=25, prompt_tokens=60, total_tokens=85, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [6]:
completion.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_cHPme3F2si8XlqzURMpW5FZA', function=Function(arguments='{"latitude":23.8103,"longitude":90.4125}', name='get_weather'), type='function')]

In [7]:
import json

tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
result

25.7

In [8]:
json.loads(completion.choices[0].message.json())


<ipython-input-8-e6b4781e58db>:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  json.loads(completion.choices[0].message.json())


{'content': None,
 'refusal': None,
 'role': 'assistant',
 'audio': None,
 'function_call': None,
 'tool_calls': [{'id': 'call_cHPme3F2si8XlqzURMpW5FZA',
   'function': {'arguments': '{"latitude":23.8103,"longitude":90.4125}',
    'name': 'get_weather'},
   'type': 'function'}]}

In [9]:
messages.append(json.loads(completion.choices[0].message.json()))  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id":tool_call.id,
    "content": str(result)
})

messages

<ipython-input-9-3c5e0c033374>:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  messages.append(json.loads(completion.choices[0].message.json()))  # append model's function call message


[{'role': 'user', 'content': "What's the weather like in Dhaka today?"},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_cHPme3F2si8XlqzURMpW5FZA',
    'function': {'arguments': '{"latitude":23.8103,"longitude":90.4125}',
     'name': 'get_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_cHPme3F2si8XlqzURMpW5FZA',
  'content': '25.7'}]

In [10]:
completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

completion_2.choices[0].message.content

'The current temperature in Dhaka is 25.7°C.'

# other example

In [11]:
messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

completion = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

In [12]:
tool_call = completion.choices[0].message.tool_calls[0]
args = json.loads(tool_call.function.arguments)

result = get_weather(args["latitude"], args["longitude"])
result

6.9

In [13]:
messages.append(json.loads(completion.choices[0].message.json()))  # append model's function call message
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id":tool_call.id,
    "content": str(result)
})

messages

<ipython-input-13-3c5e0c033374>:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  messages.append(json.loads(completion.choices[0].message.json()))  # append model's function call message


[{'role': 'user', 'content': "What's the weather like in Paris today?"},
 {'content': None,
  'refusal': None,
  'role': 'assistant',
  'audio': None,
  'function_call': None,
  'tool_calls': [{'id': 'call_YL0btM555ibXRh9ZseUDyQOb',
    'function': {'arguments': '{"latitude":48.8566,"longitude":2.3522}',
     'name': 'get_weather'},
    'type': 'function'}]},
 {'role': 'tool',
  'tool_call_id': 'call_YL0btM555ibXRh9ZseUDyQOb',
  'content': '6.9'}]

In [14]:
completion_2 = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)

completion_2.choices[0].message.content

'The current temperature in Paris is 6.9°C.'